In [66]:
import pandas as pd
import numpy as np

In [67]:
csv_link = r"C:\Users\arnau\Documents\DSTI\ML_project\data\ibtracs.csv"

In [68]:
df_ibtracs = pd.read_csv(
    csv_link,
    skiprows=[1],
    header=0,
    low_memory=False
)

In [69]:
df_ibtracs.shape

(297098, 174)

When we look at the CSV file, it seems there are a LOT of missing values.

In [70]:
df_ibtracs.isnull().sum().sum()

np.int64(73164)

73,164 missing values out of 51,695,052 cells is really small compared to how the CSV file looks. Let's take a column with a lot of missing values.

We will look at "WMO_WIND"; it's an arbitrary choice.

In [71]:
df_ibtracs.WMO_WIND.isnull().sum()

np.int64(0)

In [72]:
df_ibtracs.WMO_WIND

0          
1          
2          
3          
4          
         ..
297093     
297094     
297095     
297096     
297097     
Name: WMO_WIND, Length: 297098, dtype: object

In [73]:
df_ibtracs.WMO_WIND[0]

' '

So, there were no missing values. But when we look at the data, there are empty cells. Those cells are filled with a space character: " "

The next command will replace all the " " with null values and will also assign a numeric type to all columns with numerical values.

In [74]:
for col in df_ibtracs.columns:
    df_ibtracs[col] = df_ibtracs[col].replace(" ", np.nan)
    try:
        df_ibtracs[col] = pd.to_numeric(df_ibtracs[col])
    except ValueError:
        pass

C:\Users\arnau\AppData\Local\Temp\ipykernel_20560\8097902.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ibtracs[col] = df_ibtracs[col].replace(" ", np.nan)


In [75]:
df_ibtracs.isnull().sum().sum()

np.int64(41437829)

From 73,164 null values to 41,437,829, it represents 80% of null values.

We are looking at our target value to check the number of null values.

In [76]:
df_ibtracs.TD9636_STAGE.isnull().sum()

np.int64(248755)

Out of 297,098 rows, we have 248,755 null cells, leaving only 48,343 usable cells.

The goal of the next commands is to find an element that allows us to differentiate the cyclones.

In [77]:
df_ibtracs.SID.describe()

count            297098
unique             4767
top       1997013S08101
freq                352
Name: SID, dtype: object

In [78]:
df_ibtracs.NAME.describe()

count      297098
unique       1535
top       UNNAMED
freq        36655
Name: NAME, dtype: object

Unique values in SID = 4767 — Unique values in NAME = 1535.

So, there is probably a reuse of the same name for different SID

In [79]:
df_ibtracs[df_ibtracs['NAME'] == "PENI"]["SID"].unique()

array(['1980001S13173', '1990043S09206'], dtype=object)

For the name "PENI", we can see 2 different SID. So, we will only use the SID column to look at the precise storm.

We want to see if for the same SID, the agency TD9636 has not provided any STAGE, has partially provided a STAGE, or if the agency is present for the entire SID.

In [80]:
df_ibtracs["SID"].unique()

array(['1980001S13173', '1980002S15081', '1980003S15137', ...,
       '2024276S10076', '2024278N21265', '2024280N18146'], dtype=object)

In [81]:
null_and_valid = df_ibtracs.groupby('SID').filter(
    lambda group: group['TD9636_STAGE'].isnull().any() and group['TD9636_STAGE'].notnull().any()
)['SID'].nunique()

null = df_ibtracs.groupby('SID').filter(
    lambda group: group['TD9636_STAGE'].isnull().all()
)['SID'].nunique()

valid = df_ibtracs.groupby('SID').filter(
    lambda group: group['TD9636_STAGE'].notnull().all()
)['SID'].nunique()

In [82]:
print(f"{null_and_valid} SID have both null and valid values for TD9636_STAGE")
print(f"{null} SID have only null values for TD9636_STAGE")
print(f"{valid} SID have only valid values for TD9636_STAGE")

436 SID have both null and valid values for TD9636_STAGE
3974 SID have only null values for TD9636_STAGE
357 SID have only valid values for TD9636_STAGE


So, there are 3,974 SID that will not be usable because there is no link between these SID and our agency TD9636. There are 357 SID that are usable without any work, and 436 SID where the agency TD9636 has partially handled the stage.

We can remove the SID that have only null values in our TD9636_STAGE column.

In [83]:
sid_to_remove = df_ibtracs.groupby('SID').filter(
    lambda group: group['TD9636_STAGE'].isnull().all()
)['SID'].unique()
df_transformed = df_ibtracs.drop(df_ibtracs[df_ibtracs['SID'].isin(sid_to_remove)].index, inplace=False)

In [84]:
df_transformed.shape


(55373, 174)

To avoid duplicates, we keep only the aggregation of the "LAT" and "LON" columns

In [85]:
df_transformed.filter(regex='LAT|LON').columns

Index(['LAT', 'LON', 'USA_LAT', 'USA_LON', 'TOKYO_LAT', 'TOKYO_LON',
       'TOKYO_R50_LONG', 'TOKYO_R30_LONG', 'CMA_LAT', 'CMA_LON', 'HKO_LAT',
       'HKO_LON', 'KMA_LAT', 'KMA_LON', 'KMA_R50_LONG', 'KMA_R30_LONG',
       'NEWDELHI_LAT', 'NEWDELHI_LON', 'REUNION_LAT', 'REUNION_LON', 'BOM_LAT',
       'BOM_LON', 'NADI_LAT', 'NADI_LON', 'WELLINGTON_LAT', 'WELLINGTON_LON',
       'DS824_LAT', 'DS824_LON', 'TD9636_LAT', 'TD9636_LON', 'TD9635_LAT',
       'TD9635_LON', 'NEUMANN_LAT', 'NEUMANN_LON', 'MLC_LAT', 'MLC_LON'],
      dtype='object')

In [86]:
df_transformed = df_transformed.drop(columns=df_transformed.filter(like='_LAT').columns)
df_transformed = df_transformed.drop(columns=df_transformed.filter(like='_LON').columns)

In [87]:
df_transformed.filter(regex='LAT|LON').columns

Index(['LAT', 'LON'], dtype='object')

In [91]:
df_transformed.shape

(55373, 140)

We will use the IFLAG column, which allows us to know which agency worked on the cyclone.

IFLAG :
a 15 character flag string which denotes the source of each agency's report:
- Interpolation Flags include:
    - _ == missing reports. No information provided.
    - O == original report as provided by the agency.
    - P == position was interpolated (all variables were interpolated/filled, including intensity)
    - I == Position was provided, but Intensity variables (and likely other variables) were interpolated/filled
    - V = Position and intensity variables are original but some variables were interpolated/filled.

- The order of the 15 characters refers to the following 15 datasets:
    - 1 - USA Agency (see column 18)
    - 2 - Tokyo
    - 3 - CMA
    - 4 - HKO
    - 5 - KMA
    - 6 - NewDelhi
    - 7 - Reunion
    - 8 - BoM
    - 9 - Nadi
    - 10 - Wellington
    - 11 - ds824
    - 12 - TD9636
    - 13 - TD9635
    - 14 - Neumann Southern Hemisphere data set
    - 15 - M.L. Chenoweth N Atlantic Historic dataset

The goal is to create an average of the measurement data collected by the agencies that worked on the same cyclone.

In [94]:
df_transformed.IFLAG[0]

'O_________OO_O_'

In [98]:
df_transformed['IFLAG'].apply(lambda x: pd.Series(list(x))).stack().value_counts()

_    652252
P     99936
O     78293
V       106
I         8
Name: count, dtype: int64

We will only retrieve the data when "O == original report as provided by the agency" because extrapolation is not of interest if an original report exists.

Now that the SIDs we cannot use have been removed, the following command will allow us to delete the columns that do not contain any values.

In [89]:
'''df_transformed = df_transformed.drop(df_transformed.columns[df_transformed.isnull().all()], axis=1)'''

'df_transformed = df_transformed.drop(df_transformed.columns[df_transformed.isnull().all()], axis=1)'

In [90]:
'''df_transformed.shape'''

'df_transformed.shape'